<a href="https://colab.research.google.com/github/Pulisai1704/Software-engineering-lab/blob/main/Unit_1_8puzzle_%2C_wumpus%2C_vaccum_cleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import heapq
import numpy as np

# Directions for moving tiles (up, down, left, right)
DIRECTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]

class Node:
    def __init__(self, puzzle, blank_pos, g, h, parent=None):
        self.puzzle = puzzle
        self.blank_pos = blank_pos
        self.g = g  # Cost from start
        self.h = h  # Heuristic cost
        self.parent = parent

    def __lt__(self, other):
        return (self.g + self.h) < (other.g + other.h)

    def get_f(self):
        return self.g + self.h

def manhattan_distance(puzzle):
    distance = 0
    size = len(puzzle)
    for r in range(size):
        for c in range(size):
            value = puzzle[r][c]
            if value != 0:
                target_r, target_c = divmod(value - 1, size)
                distance += abs(target_r - r) + abs(target_c - c)
    return distance

def is_solvable(puzzle):
    one_d = [tile for row in puzzle for tile in row if tile != 0]
    inversions = 0
    for i in range(len(one_d) - 1):
        for j in range(i + 1, len(one_d)):
            if one_d[i] > one_d[j]:
                inversions += 1
    return inversions % 2 == 0

def generate_moves(node):
    neighbors = []
    size = len(node.puzzle)
    r, c = node.blank_pos

    for dr, dc in DIRECTIONS:
        nr, nc = r + dr, c + dc
        if 0 <= nr < size and 0 <= nc < size:
            new_puzzle = [row[:] for row in node.puzzle]
            new_puzzle[r][c], new_puzzle[nr][nc] = new_puzzle[nr][nc], new_puzzle[r][c]
            new_blank_pos = (nr, nc)
            neighbors.append(Node(new_puzzle, new_blank_pos, node.g + 1, manhattan_distance(new_puzzle), node))

    return neighbors

def print_puzzle(puzzle):
    for row in puzzle:
        print(" ".join(str(tile) for tile in row))
    print()

def solve_puzzle(start_puzzle):
    size = len(start_puzzle)

    if not is_solvable(start_puzzle):
        print("The puzzle is not solvable.")
        return

    start_blank_pos = next((r, c) for r, row in enumerate(start_puzzle) for c, tile in enumerate(row) if tile == 0)
    start_node = Node(start_puzzle, start_blank_pos, 0, manhattan_distance(start_puzzle))

    open_list = []
    heapq.heappush(open_list, start_node)

    closed_list = set()
    closed_list.add(str(start_puzzle))

    while open_list:
        current_node = heapq.heappop(open_list)

        if current_node.h == 0:
            print_solution(current_node)
            return

        for neighbor in generate_moves(current_node):
            state_str = str(neighbor.puzzle)
            if state_str not in closed_list:
                closed_list.add(state_str)
                heapq.heappush(open_list, neighbor)

    print("No solution found.")

def print_solution(node):
    if node is None:
        return
    print_solution(node.parent)
    print_puzzle(node.puzzle)

if __name__ == "__main__":
    puzzle = [
        [1, 2, 3],
        [4, 5, 6],
        [7, 0, 8]
    ]

    solve_puzzle(puzzle)


1 2 3
4 5 6
7 0 8

1 2 3
4 5 6
7 8 0



In [3]:
import random

# Constants
NUM_ROWS = 4
NUM_COLS = 4
NUM_PITS = 3
NUM_ARROWS = 1

# Directions
UP = (-1, 0)
DOWN = (1, 0)
LEFT = (0, -1)
RIGHT = (0, 1)

class WumpusWorld:
    def __init__(self):
        self.grid = [[' ' for _ in range(NUM_COLS)] for _ in range(NUM_ROWS)]
        self.agent_position = (0, 0)
        self.gold_position = None
        self.wumpus_position = None
        self.pits = []
        self.place_elements()

    def place_elements(self):
        self.gold_position = (random.randint(0, NUM_ROWS-1), random.randint(0, NUM_COLS-1))
        self.grid[self.gold_position[0]][self.gold_position[1]] = 'G'

        self.wumpus_position = (random.randint(0, NUM_ROWS-1), random.randint(0, NUM_COLS-1))
        while self.wumpus_position == self.gold_position:
            self.wumpus_position = (random.randint(0, NUM_ROWS-1), random.randint(0, NUM_COLS-1))
        self.grid[self.wumpus_position[0]][self.wumpus_position[1]] = 'W'

        self.pits = []
        while len(self.pits) < NUM_PITS:
            pit_position = (random.randint(0, NUM_ROWS-1), random.randint(0, NUM_COLS-1))
            if pit_position != self.gold_position and pit_position != self.wumpus_position and pit_position not in self.pits:
                self.pits.append(pit_position)
                self.grid[pit_position[0]][pit_position[1]] = 'P'

    def display(self):
        for row in self.grid:
            print(' '.join(row))
        print(f'Agent is at {self.agent_position}')

    def move_agent(self, direction):
        new_position = (self.agent_position[0] + direction[0], self.agent_position[1] + direction[1])
        if 0 <= new_position[0] < NUM_ROWS and 0 <= new_position[1] < NUM_COLS:
            self.agent_position = new_position
            self.check_status()
        else:
            print("Move out of bounds!")

    def check_status(self):
        if self.agent_position == self.gold_position:
            print("You found the gold! You win!")
            return True
        if self.agent_position in self.pits:
            print("You fell into a pit! Game over!")
            return True
        if self.agent_position == self.wumpus_position:
            print("The Wumpus got you! Game over!")
            return True
        return False

# Example usage
world = WumpusWorld()
world.display()

# Simple agent actions
world.move_agent(RIGHT)
world.move_agent(DOWN)
world.move_agent(DOWN)
world.move_agent(RIGHT)
world.move_agent(RIGHT)

world.display()


    P W
      P
G   P  
       
Agent is at (0, 0)
You fell into a pit! Game over!
    P W
      P
G   P  
       
Agent is at (2, 3)


In [4]:
import random

class VacuumCleanerWorld:
    def __init__(self):
        # Initialize the rooms as either 'Clean' or 'Dirty'
        self.rooms = {'Room1': 'Dirty', 'Room2': 'Dirty'}
        # Initialize the vacuum cleaner's position (start in Room1)
        self.vacuum_position = 'Room1'
        # Track the number of cleaning actions
        self.cleaning_actions = 0

    def display(self):
        # Display the current state of the rooms and the vacuum cleaner's position
        print(f"Vacuum Position: {self.vacuum_position}")
        print(f"Room1: {self.rooms['Room1']}")
        print(f"Room2: {self.rooms['Room2']}")
        print(f"Number of cleaning actions: {self.cleaning_actions}")

    def move(self, room):
        if room in self.rooms:
            self.vacuum_position = room
            print(f"Moved to {room}")
        else:
            print("Invalid room!")

    def clean(self):
        if self.vacuum_position in self.rooms:
            if self.rooms[self.vacuum_position] == 'Dirty':
                self.rooms[self.vacuum_position] = 'Clean'
                self.cleaning_actions += 1
                print(f"Cleaned {self.vacuum_position}")
            else:
                print(f"{self.vacuum_position} is already clean")
        else:
            print("Invalid room to clean!")

    def perform_action(self, action):
        # Perform an action based on the given command
        if action == 'clean':
            self.clean()
        elif action.startswith('move'):
            _, room = action.split()
            self.move(room)
        else:
            print("Invalid action!")

# Example usage
world = VacuumCleanerWorld()
world.display()

# Example actions
world.perform_action('clean')  # Clean the current room (Room1)
world.perform_action('move Room2')  # Move to Room2
world.perform_action('clean')  # Clean the new room (Room2)

world.display()


Vacuum Position: Room1
Room1: Dirty
Room2: Dirty
Number of cleaning actions: 0
Cleaned Room1
Moved to Room2
Cleaned Room2
Vacuum Position: Room2
Room1: Clean
Room2: Clean
Number of cleaning actions: 2
